In [205]:
import pandas as pd
import ast
import os
from datetime import datetime
from dateutil import parser
from collections import defaultdict
base = "ivory"
df = pd.read_csv(f'{base}/{base}.csv')

In [206]:
df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count
0,sarina,445,[{'tweet_link': '/sariverse/status/17945827511...,@sariverse,"1,414","17,208",[{'tweet_link': '/sariverse/status/17944598638...,2:46 PM - 21 Aug 2012,"16,380"
1,Allison Blake,44,[{'tweet_link': '/cryptoAllis0n/status/1794626...,@cryptoAllis0n,5,117,[{'tweet_link': '/cryptoAllis0n/status/1794503...,9:55 PM - 13 Jul 2023,148
2,Dazed,"9,691",[{'tweet_link': '/Dazed/status/179146452872461...,@Dazed,"1,220,305","2,741",[{'tweet_link': '/Dazed/status/175452559032805...,1:38 PM - 4 Jul 2008,"17,456"
3,The Billowing Hilltop,888,[{'tweet_link': '/Billowinghillto/status/17944...,@Billowinghillto,"1,435","14,758",[{'tweet_link': '/Billowinghillto/status/16388...,10:38 AM - 26 Apr 2019,"8,952"
4,Muffin me.,"1,133",[{'tweet_link': '/SilentTheatre1/status/179412...,@SilentTheatre1,"1,202","54,388",[{'tweet_link': '/SilentTheatre1/status/176500...,12:59 PM - 13 Oct 2011,"90,796"


In [207]:
##1. Convert join date to unix join date
def convert_to_unix_join_date(timestamp):
    # Parse the timestamp
    dt = pd.to_datetime(timestamp, format='%I:%M %p - %d %b %Y')
    # Extract the date part and reset time to midnight
    dt_date_only = dt.normalize()
    # Convert to Unix timestamp (seconds since epoch)
    unix_timestamp = int(dt_date_only.timestamp())
    return unix_timestamp


In [208]:
df['unix_join_date'] = df['join_date'].apply(convert_to_unix_join_date)

In [209]:
df.head()


,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date
0,sarina,445,[{'tweet_link': '/sariverse/status/17945827511...,@sariverse,"1,414","17,208",[{'tweet_link': '/sariverse/status/17944598638...,2:46 PM - 21 Aug 2012,"16,380",1345507200
1,Allison Blake,44,[{'tweet_link': '/cryptoAllis0n/status/1794626...,@cryptoAllis0n,5,117,[{'tweet_link': '/cryptoAllis0n/status/1794503...,9:55 PM - 13 Jul 2023,148,1689206400
2,Dazed,"9,691",[{'tweet_link': '/Dazed/status/179146452872461...,@Dazed,"1,220,305","2,741",[{'tweet_link': '/Dazed/status/175452559032805...,1:38 PM - 4 Jul 2008,"17,456",1215129600
3,The Billowing Hilltop,888,[{'tweet_link': '/Billowinghillto/status/17944...,@Billowinghillto,"1,435","14,758",[{'tweet_link': '/Billowinghillto/status/16388...,10:38 AM - 26 Apr 2019,"8,952",1556236800
4,Muffin me.,"1,133",[{'tweet_link': '/SilentTheatre1/status/179412...,@SilentTheatre1,"1,202","54,388",[{'tweet_link': '/SilentTheatre1/status/176500...,12:59 PM - 13 Oct 2011,"90,796",1318464000


In [210]:
##2. convert last tweet date to unix last tweet date

def convert(row):
    return ast.literal_eval(row['tweets'])
df['ptweets'] = df.apply(convert,axis = 1)
df['preplies'] = df.apply(convert,axis = 1)

In [211]:
def parse_date(date_str):
    return datetime.strptime(date_str, '%b %d, %Y · %I:%M %p %Z')

In [212]:
#sort the tweets and replies by 
def get_latest_first_entry(row):
    first_tweet_date = None
    first_reply_date = None
    if len(row['ptweets'])!=0:
        temp = row['ptweets']
        latest_date = None        
        for tweet in temp:
            tweet_date = parse_date(tweet['tweet_date'])
            if latest_date is None or tweet_date > latest_date:
                latest_date = tweet_date
        first_tweet_date = latest_date
    if len(row['preplies'])!=0:
        temp = row['preplies']
        latest_date = None        
        for tweet in temp:
            tweet_date = parse_date(tweet['tweet_date'])
            if latest_date is None or tweet_date > latest_date:
                latest_date = tweet_date
        first_reply_date = latest_date
    
    if first_tweet_date and first_reply_date:
        return first_tweet_date if first_tweet_date > first_reply_date else first_reply_date
    elif first_tweet_date:
        return first_tweet_date
    elif first_reply_date:
        return first_reply_date
    else:
        return None


def convert_date_to_unix(timestamp):
    dt = pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S')
    
    dt_date_only = dt.normalize()
    
    unix_timestamp = int(dt_date_only.timestamp())
    return unix_timestamp

df['last_tweet_date'] = df.apply(get_latest_first_entry, axis=1)
df = df[df['last_tweet_date'].notnull()]
df['unix_last_tweet_date'] = df['last_tweet_date'].apply(convert_date_to_unix)


In [213]:
df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,ptweets,preplies,last_tweet_date,unix_last_tweet_date
0,sarina,445,[{'tweet_link': '/sariverse/status/17945827511...,@sariverse,"1,414","17,208",[{'tweet_link': '/sariverse/status/17944598638...,2:46 PM - 21 Aug 2012,"16,380",1345507200,[{'tweet_link': '/sariverse/status/17944598638...,[{'tweet_link': '/sariverse/status/17944598638...,2024-05-25 20:05:00,1716595200
1,Allison Blake,44,[{'tweet_link': '/cryptoAllis0n/status/1794626...,@cryptoAllis0n,5,117,[{'tweet_link': '/cryptoAllis0n/status/1794503...,9:55 PM - 13 Jul 2023,148,1689206400,[{'tweet_link': '/cryptoAllis0n/status/1794503...,[{'tweet_link': '/cryptoAllis0n/status/1794503...,2024-05-25 23:00:00,1716595200
2,Dazed,"9,691",[{'tweet_link': '/Dazed/status/179146452872461...,@Dazed,"1,220,305","2,741",[{'tweet_link': '/Dazed/status/175452559032805...,1:38 PM - 4 Jul 2008,"17,456",1215129600,[{'tweet_link': '/Dazed/status/175452559032805...,[{'tweet_link': '/Dazed/status/175452559032805...,2024-05-25 08:00:00,1716595200
3,The Billowing Hilltop,888,[{'tweet_link': '/Billowinghillto/status/17944...,@Billowinghillto,"1,435","14,758",[{'tweet_link': '/Billowinghillto/status/16388...,10:38 AM - 26 Apr 2019,"8,952",1556236800,[{'tweet_link': '/Billowinghillto/status/16388...,[{'tweet_link': '/Billowinghillto/status/16388...,2024-05-25 10:17:00,1716595200
4,Muffin me.,"1,133",[{'tweet_link': '/SilentTheatre1/status/179412...,@SilentTheatre1,"1,202","54,388",[{'tweet_link': '/SilentTheatre1/status/176500...,12:59 PM - 13 Oct 2011,"90,796",1318464000,[{'tweet_link': '/SilentTheatre1/status/176500...,[{'tweet_link': '/SilentTheatre1/status/176500...,2024-05-25 16:38:00,1716595200


In [214]:
df['difference_date']=df['unix_last_tweet_date']-df['unix_join_date']
df['difference_date']=df['difference_date']/(3600*24)


In [215]:
#new_df=df[df['difference_date']<=360]
#new_df=new_df.copy()
new_df=df.copy()

In [216]:
print(len(new_df))
new_df.dtypes

6778


username                        object
following_count                 object
replies                         object
handle                          object
followers_count                 object
likes_count                     object
tweets                          object
join_date                       object
tweet_count                     object
unix_join_date                   int64
ptweets                         object
preplies                        object
last_tweet_date         datetime64[ns]
unix_last_tweet_date             int64
difference_date                float64
dtype: object

In [217]:
import numpy as np
# I need to sort the dataframe according to posting frequency
new_df.loc[:,'tweet_count'] = new_df['tweet_count'].astype(str).str.replace(',', '').astype(float)

# Replace zero values in difference_date column with NaN
new_df.loc[:,'difference_date'] = new_df['difference_date'].replace(0, np.nan)

# Fill NaN values with a default value (e.g., 1) to avoid division by zero
new_df.loc[:,'difference_date'] = new_df['difference_date'].fillna(1)

# Perform division to calculate post_freq
new_df.loc[:,'post_freq'] = new_df['tweet_count'] / new_df['difference_date']

# Sort the DataFrame by post_freq column
new_df = new_df.sort_values(by='post_freq', ascending=False)
# new_df.loc[:,'post_freq'] = new_df['tweet_count']/new_df['difference_date']
# new_df.head()
# df_sorted = new_df.sort_values(by = 'post_freq', ascending=False)

In [218]:
new_df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,ptweets,preplies,last_tweet_date,unix_last_tweet_date,difference_date,post_freq
897,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,[{'tweet_link': '/ArmandKleinX/status/15951495...,[{'tweet_link': '/ArmandKleinX/status/15951495...,2022-11-22 20:17:00,1669075200,2.0,126525.5
3133,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,2021-04-03 13:39:00,1617408000,1.0,35827.0
1887,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,[{'tweet_link': '/eenopslaghier/status/1518860...,[{'tweet_link': '/eenopslaghier/status/1518860...,2022-04-26 07:51:00,1650931200,1.0,7559.0
2725,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,[{'tweet_link': '/born2_wild/status/9132185980...,[{'tweet_link': '/born2_wild/status/9132185980...,2017-09-28 01:47:00,1506556800,13.0,2139.923077
5758,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,[{'tweet_link': '/McPepes_AI/status/1680978593...,[{'tweet_link': '/McPepes_AI/status/1680978593...,2023-07-17 16:31:00,1689552000,213.0,1954.619718


In [219]:
len(new_df)

6778

In [220]:
def parse_date(date_str):
    return datetime.strptime(date_str, '%b %d, %Y · %I:%M %p %Z')
def convert_date_to_unix(timestamp):
    
    dt = pd.to_datetime(parse_date(timestamp), format='%Y-%m-%d %H:%M:%S')
    
    dt_date_only = dt.normalize()
    
    unix_timestamp = int(dt_date_only.timestamp())
    return unix_timestamp

In [221]:
def make_time_series(row):
    temp = row['ptweets'] + row['preplies']
    sorted_tweets = sorted(temp, key=lambda x: convert_date_to_unix(x['tweet_date']))
    time_series = []
    for tweet in sorted_tweets:
        time_series.append(convert_date_to_unix(tweet['tweet_date']))
    tweets_by_date = defaultdict(int)
    for tweet in sorted_tweets:
        tweet_date_unix = convert_date_to_unix(tweet['tweet_date'])
        tweets_by_date[tweet_date_unix] += 1
    
    # Convert to array of tuples
    tweets_count_array = [(date, count) for date, count in tweets_by_date.items()]
    sorted_tweets_count_array = sorted(tweets_count_array, key=lambda x: x[0])
    return sorted_tweets_count_array

new_df['time_series'] = new_df.apply(make_time_series, axis = 1)

In [222]:
new_df.head()
#to_csv  ##SAVE

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,ptweets,preplies,last_tweet_date,unix_last_tweet_date,difference_date,post_freq,time_series
897,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,[{'tweet_link': '/ArmandKleinX/status/15951495...,[{'tweet_link': '/ArmandKleinX/status/15951495...,2022-11-22 20:17:00,1669075200,2.0,126525.5,"[(1669075200, 2)]"
3133,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,2021-04-03 13:39:00,1617408000,1.0,35827.0,"[(1617408000, 2)]"
1887,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,[{'tweet_link': '/eenopslaghier/status/1518860...,[{'tweet_link': '/eenopslaghier/status/1518860...,2022-04-26 07:51:00,1650931200,1.0,7559.0,"[(1650931200, 2)]"
2725,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,[{'tweet_link': '/born2_wild/status/9132185980...,[{'tweet_link': '/born2_wild/status/9132185980...,2017-09-28 01:47:00,1506556800,13.0,2139.923077,"[(1506556800, 2)]"
5758,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,[{'tweet_link': '/McPepes_AI/status/1680978593...,[{'tweet_link': '/McPepes_AI/status/1680978593...,2023-07-17 16:31:00,1689552000,213.0,1954.619718,"[(1689552000, 2)]"


In [223]:
import numpy as np

# Function to calculate common statistical quantities
def calculate_common_statistics(time_series):
    # Extract counts from time series
    try:
        counts = [count for _, count in time_series]
        
        # Calculate statistical quantities
        statistics = {
            'mean': np.mean(counts),
            'median': np.median(counts),
            'std_dev': np.std(counts),
            'min': np.min(counts),
            'max': np.max(counts),
            '25th_percentile': np.percentile(counts, 25),
            '75th_percentile': np.percentile(counts, 75),
            # Add more statistical quantities as needed
        }
        
        return statistics
    except:
        return {
            'mean': None,
            'median': None,
            'std_dev': None,
            'min': None,
            'max': None,
            '25th_percentile': None,
            '75th_percentile': None,
            # Add more statistical quantities as needed
        }

# Apply the function to each row in the DataFrame
statistics = new_df['time_series'].apply(calculate_common_statistics)

# Convert the list of dictionaries to a DataFrame
statistics_df = pd.DataFrame(statistics.tolist())



In [224]:
new_df = new_df.reset_index(drop = True)
new_df = pd.concat([new_df,statistics_df],axis = 1)
new_df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,...,difference_date,post_freq,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile
0,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,...,2.0,126525.5,"[(1669075200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
1,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,...,1.0,35827.0,"[(1617408000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
2,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,...,1.0,7559.0,"[(1650931200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
3,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,...,13.0,2139.923077,"[(1506556800, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
4,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,...,213.0,1954.619718,"[(1689552000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0


In [225]:
new_df = new_df.dropna(subset=['handle'])
columns_to_drop = ['mean', 'median', 'std_dev', 'min', 'max', '25th_percentile', '75th_percentile']
new_df = new_df.dropna(subset=columns_to_drop)
new_df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,...,difference_date,post_freq,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile
0,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,...,2.0,126525.5,"[(1669075200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
1,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,...,1.0,35827.0,"[(1617408000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
2,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,...,1.0,7559.0,"[(1650931200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
3,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,...,13.0,2139.923077,"[(1506556800, 2)]",2.0,2.0,0.0,2,2,2.0,2.0
4,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,...,213.0,1954.619718,"[(1689552000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0


In [226]:
len(new_df)

6778

In [227]:
def find_num_scraped_tweets(row):
    temp = row['ptweets'] + row['preplies']
    return len(temp)    
new_df['num_scraped_tweets'] = new_df.apply(find_num_scraped_tweets, axis = 1)
new_df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,...,post_freq,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile,num_scraped_tweets
0,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,...,126525.5,"[(1669075200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
1,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,...,35827.0,"[(1617408000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
2,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,...,7559.0,"[(1650931200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
3,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,...,2139.923077,"[(1506556800, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
4,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,...,1954.619718,"[(1689552000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2


In [228]:
new_df.to_csv(f'{base}/{base}_feature_engineered.csv')

In [229]:
new_df.head()

,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,unix_join_date,...,post_freq,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile,num_scraped_tweets
0,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,1668902400,...,126525.5,"[(1669075200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
1,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,1617408000,...,35827.0,"[(1617408000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
2,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,1650931200,...,7559.0,"[(1650931200, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
3,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,1505433600,...,2139.923077,"[(1506556800, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2
4,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,1671148800,...,1954.619718,"[(1689552000, 2)]",2.0,2.0,0.0,2,2,2.0,2.0,2


In [230]:
df = pd.read_csv(f'{base}/{base}_feature_engineered.csv')

In [231]:
import ast
def stdify(row):
    time_series = row['time_series']
    temp = ast.literal_eval(time_series)
    total_time = 0.0
    total_count=0
    weighted_sum_of_time_and_count=0.0
    for point in temp:
        total_time += point[0]
        total_count += point[1]
        weighted_sum_of_time_and_count += point[0]* point[0]
    # Calculate mean (weighted average)
    mean = weighted_sum_of_time_and_count / total_count

    # Calculate squared deviations from the mean
    squared_deviations = [((point[0] - mean) * point[1])**2 for point in temp]

    # Calculate variance (average of squared deviations)
    variance = sum(squared_deviations) / total_count

    # Calculate standard deviation (square root of variance)
    std = variance**0.5
    return std
        
df['std_dev'] = df.apply(stdify,axis=1)

In [232]:
#calculate average tweet length
def calc_tweet_length(row):
    tweets = ast.literal_eval(row['tweets'])
    replies = ast.literal_eval(row['replies'])
    temp = tweets+replies
    num_tweets = len(temp)
    total_length=0
    for tweet in temp:
        total_length+=len(tweet['tweet_content'].strip())
    return total_length/num_tweets
df['avg_tweet_length'] = df.apply(calc_tweet_length,axis=1)
df.head()

,Unnamed: 0,username,following_count,replies,handle,followers_count,likes_count,tweets,join_date,tweet_count,...,time_series,mean,median,std_dev,min,max,25th_percentile,75th_percentile,num_scraped_tweets,avg_tweet_length
0,0,Armand Klein,"58,751",[],@ArmandKleinX,"59,120","281,485",[{'tweet_link': '/ArmandKleinX/status/15951495...,5:57 PM - 20 Nov 2022,253051.0,...,"[(1669075200, 2)]",2.0,2.0,1.969867e+18,2,2,2.0,2.0,2,160.000000
1,1,3rd Time's A Charm,"3,074",[{'tweet_link': '/ThirdTimeIsACh2/status/17952...,@ThirdTimeIsACh2,"2,444","38,585",[{'tweet_link': '/ThirdTimeIsACh2/status/13783...,1:33 PM - 3 Apr 2021,35827.0,...,"[(1617408000, 2)]",2.0,2.0,1.849797e+18,2,2,2.0,2.0,2,99.275510
2,2,enn 🐮,"1,763",[{'tweet_link': '/eenopslaghier/status/1795132...,@eenopslaghier,"1,281","13,643",[{'tweet_link': '/eenopslaghier/status/1518860...,7:03 AM - 26 Apr 2022,7559.0,...,"[(1650931200, 2)]",2.0,2.0,1.927272e+18,2,2,2.0,2.0,2,6.250000
3,3,Born2BWild,357,[{'tweet_link': '/born2_wild/status/1780550740...,@born2_wild,"1,007","13,840",[{'tweet_link': '/born2_wild/status/9132185980...,12:49 AM - 15 Sep 2017,27819.0,...,"[(1506556800, 2)]",2.0,2.0,1.604930e+18,2,2,2.0,2.0,2,49.750000
4,4,McPepe's,57,[{'tweet_link': '/McPepes_AI/status/1795606295...,@McPepes_AI,"90,021",532,[{'tweet_link': '/McPepes_AI/status/1680978593...,3:05 AM - 16 Dec 2022,416334.0,...,"[(1689552000, 2)]",2.0,2.0,2.018497e+18,2,2,2.0,2.0,2,32.405941


In [233]:
columns = ['ptweets', 'preplies']
df=df.drop(columns,axis=1)

In [234]:
df.dtypes

Unnamed: 0                int64
username                 object
following_count          object
replies                  object
handle                   object
followers_count          object
likes_count              object
tweets                   object
join_date                object
tweet_count             float64
unix_join_date            int64
last_tweet_date          object
unix_last_tweet_date      int64
difference_date         float64
post_freq               float64
time_series              object
mean                    float64
median                  float64
std_dev                 float64
min                       int64
max                       int64
25th_percentile         float64
75th_percentile         float64
num_scraped_tweets        int64
avg_tweet_length        float64
dtype: object

In [235]:
df.to_csv(f'{base}/{base}_feature_engineered.csv')